In [1]:
import pandas as pd
import numpy as np

# plotting not yet supported
from cartopy import crs as ccrs, feature as cfeature 
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt

In [ ]:
prod_vol = pd.read_csv("./data/upstream_weekly-field_annual_production_kbd_2015-2023.csv")

In [2]:
new_prod_in = pd.read_csv("./data/country-climate-trace_oil-and-gas-production_100724.csv")
new_trans_in = pd.read_csv("./data/country-climate-trace_oil-and-gas-transport_100724.csv")

In [ ]:
trace22_in = pd.read_csv("./data/oil-and-gas-production-and-transport_country_emissions.csv") 

In [12]:
new_prod_in.dropna(subset=['start_date'], inplace=True)
new_prod = new_prod_in[
    new_prod_in['start_date'].str.contains('2023|2022')
]
new_prod = new_prod[
    new_prod['end_date'].str.contains('2023|2022')
]

new_trans_in.dropna(subset=['start_date'], inplace=True)
new_trans = new_trans_in[
    new_trans_in['start_date'].str.contains('2023|2022')
]
new_trans = new_trans[
    new_trans['end_date'].str.contains('2023|2022')
]

In [ ]:
()

In [25]:
import pandas as pd

# Sample dataframe
data = {
    'iso': ['US', 'US', 'UK', 'UK'],
    'start_date': ['2022-07-14', '2023-11-30', '2022-07-14', '2023-11-30'],
    'value1': [10, 30, 50, 70],
    'value2': [100, 300, 500, 700],
    'value3': [15, 35, 55, 75],
    'value4': [150, 350, 550, 750],
    'value5': [20, 40, 60, 80]
}
df = pd.DataFrame(data)

# Melting the dataframe to long format
df_long = pd.melt(df, id_vars=['iso', 'start_date'], value_vars=['value1', 'value2', 'value3', 'value4', 'value5'])

# Pivoting the dataframe
df_wide = df_long.pivot_table(index='iso', columns=['start_date', 'variable'], values='value').reset_index()

# Flattening the MultiIndex in columns
df_wide.columns = [f'{col[1]}_{col[0]}' if col[1] else col[0] for col in df_wide.columns]

df

,iso,start_date,value1,value2,value3,value4,value5
0,US,2022-07-14,10,100,15,150,20
1,US,2023-11-30,30,300,35,350,40
2,UK,2022-07-14,50,500,55,550,60
3,UK,2023-11-30,70,700,75,750,80


In [18]:
test_long = new_prod.melt(id_vars=['iso3_country','start_date'],value_vars=['CO2_emissions_tonnes','CH4_emissions_tonnes'])
test_long.pivot(index='iso3_country',columns=['start_date','iso3_country'],values='value')

ValueError: The name iso3_country occurs multiple times, use a level number

In [ ]:
# country aggregated former run


trace22 = trace22_in[['iso3_country','end_time','gas','emissions_quantity']]            # restrict to only relevant columns
trace22 = trace22[trace22['end_time'].str.contains('2022')].drop(columns=['end_time'])  # keep only items that are from 2022 run and drop time
trace22 = trace22[trace22['gas'].isin(['co2','ch4'])]                                   # restrict to only carbon dioxide and methane

# pivot table and reset index to mimic other format
trace22 = trace22.pivot(                                                                
    index='iso3_country', columns='gas', values='emissions_quantity'
).reset_index()

# give column names for the gases
trace22.columns.name = None                                                         
trace22 = trace22.rename(columns={
    'co2': 'CO2_trace22_total',
    'ch4': 'CH4_trace22_total'
})

trace22 = trace22.replace(0,np.nan)   

In [ ]:
new_prod.set_index('iso3_country', inplace=True)
new_trans.set_index('iso3_country', inplace=True)
trace22.set_index('iso3_country', inplace=True)

# Join dataframes
joined = new_prod.join(
    new_trans, how='outer',
    lsuffix='_prod',rsuffix='_trans').join(
        trace22, how='outer')

joined.reset_index(inplace=True)

In [ ]:
joined['CO2_trace_new_total'] = joined['CO2_emissions_tonnes_prod'] + joined['CO2_emissions_tonnes_trans']
joined['CH4_trace_new_total'] = joined['CH4_emissions_tonnes_prod'] + joined['CH4_emissions_tonnes_trans']

merged = joined[
    ['iso3_country',
     'CO2_emissions_tonnes_prod', 'CO2_emissions_tonnes_trans',
     'CH4_emissions_tonnes_prod', 'CH4_emissions_tonnes_trans',
     'CO2_trace_new_total', 'CH4_trace_new_total',
     'CO2_trace22_total', 'CH4_trace22_total']
]

world_row = pd.DataFrame([merged.sum()],columns=merged.columns)
world_row['iso3_country'] = 'World'
merged = pd.concat([merged,world_row], ignore_index=True)